<a href="https://colab.research.google.com/github/WKoishi/deep_learning_colab/blob/main/xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 查看当前GPU信息

In [ ]:
!/opt/bin/nvidia-smi

### 解压在本地整理好并上传的数据集

In [ ]:
import os
os.chdir('/content/drive/MyDrive')

!unzip dataset_face_shuffle.zip

### 神经网络的搭建和训练

In [6]:
import os, datetime
os.chdir('/content/drive/MyDrive')
%load_ext tensorboard

import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import callbacks
import os
import numpy as np

logdir = os.path.join("my_train_log", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks_list = [   
    # callbacks.ModelCheckpoint(   
    #     filepath='my_model.h5',   
    #     monitor='val_loss',   
    #     save_best_only=True, 
    # ), 
    callbacks.TensorBoard(
        log_dir = logdir,
    )
]

# 设置GPU显存
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

img_shape = (128, 128)
train_dir = 'dataset_face_shuffle/train'
validation_dir = 'dataset_face_shuffle/validation'

#将训练数据进行增强
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,  #图像随机旋转的角度范围
    width_shift_range=0.1,  #图像在水平或垂直方向上平移的范围（相对于高度或宽度的比例）
    height_shift_range=0.1,
    zoom_range = 0.1,  #随机缩放的范围
    horizontal_flip=True,  #随机将一半图像水平翻转
    vertical_flip=True,     #随机垂直翻转
    fill_mode='nearest',  #填充新创建的像素
)

#将验证数据进行轻度的增强
validation_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    # rotation_range=20,
    # vertical_flip=True,
    # horizontal_flip=True
)


#使用Python生成器
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = img_shape, 
    batch_size = 32,  #每个批量中包含20个样本
    class_mode ='categorical' 
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = img_shape, 
    batch_size = 16,
    class_mode = 'categorical'
)


base_model = Xception(input_shape=(128,128,3), include_top=False,
                    weights='imagenet')

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)

predictions = layers.Dense(2, name='Logits')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# for i, layer in enumerate(model.layers):
#    print(i, layer.trainable, layer.name)

#print(model.summary())


model.compile(optimizer = optimizers.Adam(learning_rate=0.0002),
            loss = losses.CategoricalCrossentropy(from_logits=True),
            metrics=['acc'])

history=model.fit(train_generator,
          steps_per_epoch=92,
          epochs=35,
          validation_data=validation_generator,
          validation_steps=46,
          callbacks=callbacks_list)

#get_tf_model_proto(model)  #保存成pb文件
model.save('FR_classify_1.h5')






The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Found 2950 images belonging to 2 classes.
Found 736 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/35
92/92 [==============================] - 47s 450ms/step - loss: 0.5791 - acc: 0.7025 - val_loss: 0.4494 - val_acc: 0.7840
Epoch 2/35
92/92 [==============================] - 40s 435ms/step - loss: 0.4399 - acc: 0.8067 - val_loss: 0.4364 - val_acc: 0.8152
Epoch 3/35
92/92 [==============================] - 40s 431ms/step - loss: 0.3950 - acc: 0.8232 - val_loss: 0.3958 - val_acc: 0.8342
Epoch 4/35
92/92 [==============================] - 40s 432ms/step - loss: 0.3522 - acc: 0.8557 - val_loss: 0.3924 - val_acc: 0.8302
Epoch 5/35
92/92 [==============================] - 40s 433ms/step - loss: 0.3330 - acc: 0.8588 - val_loss: 0.4206 - val_acc: 0.8261
Epoch 6/35
92/92 [==============================] - 40s 433ms/step - loss: 0.3117 - acc: 0.8735 - val_loss: 0.5515 - val_acc: 0.8043
Epoch 7/35
92/92 [==============================] - 40s 435ms/step - loss: 0.2948 - acc: 0.8787 - val_loss: 0.3534 - val_acc: 0.8492
Epoch 8/35
92/92 [==============================] - 40s 431ms/step - 

### 打开tensorboard查看训练记录

In [ ]:
import os
os.chdir('/content/drive/MyDrive')
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard --logdir my_train_log